# Context engineering

In [ ]:
## Static prompt

PROMPT = """
You are an expert of ...

Your task is to...

User query: {query}

Information ou could refer to:
Document summary: {summary}
Relevant memory: {memory}
"""

In [ ]:
## Context engineering

async def _execute_task(self, task_id: str, task: Dict[str, Any]) -> Dict[str, Any]:
    """Execute a task with contextual memory support."""
    
    self.active_tasks[task_id] = task
    content = task.get("content", "")
    context_data = task.get("context", {})

    # Step 1: 通过用户输入的内容（content）和已有上下文（context_data），从智能体的记忆模块中检索出相关的记忆片段。
    relevant_memories = self.memory_manager.retrieve_relevant_memories(content, context_data)

    # Step 2: Build context for the task
    context = self._build_context(task, relevant_memories)

    # Step 3: 构建一个有用的 Prompt 环境，以支持大模型生成更符合语境的输出。
    if"conversation_id"in task:
        response = await self.responses_api.continue_conversation(
            conversation_id=task["conversation_id"],
            user_input=content,
            agent_id=self.agent_card.agent_id
        )
    else:
        conv_state = await self.responses_api.create_conversation(
            user_input=content,
            system_prompt=self.custom_agent.to_system_prompt()
        )
        response = conv_state.messages[-1]["content"]

    # Step 4: Save task + response to memory
    self.memory_manager.add_episodic_memory({
        "task_id": task_id,
        "content": content,
        "response": response,
        "context": context,
        "agent_id": self.agent_card.agent_id
    })

    return {
        "task_id": task_id,
        "status": "completed",
        "response": response,
        "agent_id": self.agent_card.agent_id
    }


def _build_context(self, task: Dict[str, Any], relevant_memories: List[Dict[str, Any]]) -> Dict[str, Any]:
    """Build structured context for a task."""
    return {
        "current_task": task,
        "relevant_memories": relevant_memories,
        "agent_profile": {
            "name": self.custom_agent.name,
            "personality_type": self.custom_agent.personality_type,
            "expertise": self.custom_agent.occupation
        },
        "peer_agents": list(self.peer_agents.keys())
    }